# Credit risk assessment based on credit reports

This example illustrates how to use LLMs for credit risk assessment given a credit report. We use Meta's Llama 3.2 vision 11B as our LLM. Running this example requires a GPU with at least 24GB of memory (e.g., RTX 4090).

## Step 1: Load the model

In [1]:
from transformers import MllamaForConditionalGeneration, AutoProcessor
import torch

# To load the Llama 3.2 vision model, make sure to first login using your HuggingFace access token
# `huggingface-cli login`
model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.cuda()
processor = AutoProcessor.from_pretrained(model_id)

/home/fangzhou/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 5/5 [00:02<00:00,  1.89it/s]


## Step 2: Prepare credit report in plain text

In [3]:
import os
from PIL import Image

# Format input for multi-modal query
messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "This image shows one page of a credit report. Convert it into plain text. Do not include text not present in the image."}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)

# Translate images into plain text
## NOTE: The images are obtained from PDF
image_dir = "./data/credit_report"
report = []
for p in sorted(os.listdir(image_dir)):
    image = Image.open(os.path.join(image_dir, p))
    inputs = processor(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(model.device)

    output = model.generate(**inputs, max_new_tokens=2048)
    output = processor.decode(output[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
    report.append(output)

# Combine text from individual pages
report = "\n".join(report)
print(report)

**Credit Report Text Conversion**

**Header Section**

**TRANS UNION CREDIT REPORT REVIEW**
**Based on Report Requested on 04/16/21 at 11:35**
**MARY A MEMBER**
**01/01/1980 520-11-2222**

**Credit Score Section**

**Your Credit Score is *650*.**
**We rate your credit score as *B* and this would result in some of the conditions (such as interest rate, maximum term and loan-to-value requirements) of any loan we might offer you to be less favorable than you would receive if your credit score was higher.**
**The key factors that adversely affected your credit score are:**
**Length of time accounts have been established**
**Proportion of loan balances to loan amounts is too high**
**Length of time revolving accounts have been established**
**Proportion of balances to credit limits is too high on bank/revolving accounts**
**NOTE: The number of inquiries into your credit adversely affected your credit score.**

**Credit History Section**

**Having a combination of the different credit types 

## Step 3: Credit risk assessment

In [5]:
# Format input for text-only query
messages = [
    {"role": "user", "content": [
        {"type": "text", "text": "Comment on Mary's credit risk based on her credit report below.\n" + report}
    ]}  # Feel free to replace the text here for more specific queries
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)

# Generate credit risk assessment
inputs = processor(
    None,
    input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=2048)
output = processor.decode(output[0][len(inputs.input_ids[0]):], skip_special_tokens=True)

print(output)

Based on Mary's credit report, her credit risk can be assessed as follows:

1. **Credit Score:** Mary's credit score is 650, which is classified as "B" by the credit reporting agency. This indicates that she has some credit history, but it is not as strong as it could be.
2. **Credit History:** Mary has a mix of different credit types, including installment loans, revolving credit, and a mortgage. However, she has a high debt-to-income ratio, which could be a concern.
3. **Payment History:** Mary has a history of late payments, including a 30-day late payment on a revolving account. This could negatively impact her credit score.
4. **Public Records:** Mary has a Chapter 7 bankruptcy filing, which will remain on her credit report for up to 10 years. This is a significant negative mark on her credit report.
5. **Collections:** Mary has two collection accounts, which will also remain on her credit report for up to 7 years. These accounts are currently unpaid.
6. **Credit Utilization:** Ma